In [1]:
# Step 1: Load an experiment configuration from a JSON file
import json
import os
import glob
from scripts.sample_db import SampleDB
from scripts.config_model import Experiment, load_experiment_config, save_experiment_config, update_experiment_config


In [2]:
# Inputs
# Load database csv file
db_file_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv'

# root path 
root_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3'


In [3]:
# Loading experiment configuration
config_file_path = glob.glob(os.path.join(root_path, 'config_*.json'))[0]
print(config_file_path)

# Define your directories here
trials_path = os.path.join(root_path, "trials") 
anatomy_path = os.path.join(root_path, "anatomy") 
raw_trials_path = os.path.join(trials_path,"raw")
em_path = None  # Set to None if not applicable

config = load_experiment_config(config_file_path)
print("Loaded experiment configuration:")
print(config)


\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\config_20220426_RM0008_130hpf_fP1_f3.json
Loaded experiment configuration:
sample=Sample(id='20220426_RM0008_130hpf_fP1_f3', parents_id='P1', genotype='aTubulin:GCamp6s x GAD1b:DsRed', phenotype='positive expression, nacre', dof='21.04.2022-09:30', hpf=130, body_length_mm=4) params_odor=ParamsOdor(odor_list=['Ala', 'Ser', 'Ctrl', 'TCA', 'GCA', 'TDCA', 'Cad', 'SA'], odor_sequence=['Ala', 'Ser', 'Ctrl', 'TCA', 'GCA', 'TDCA', 'Cad', 'SA', 'Cad', 'TDCA', 'GCA', 'TCA', 'Ser', 'Ctrl', 'Ala', 'SA', 'Ctrl', 'Cad', 'Ala', 'Ser', 'TDCA', 'GCA', 'TCA', 'SA'], odor_concentration_uM=[OdorConcentration(name='Ala', concentration_mM=100.0), OdorConcentration(name='Ser', concentration_mM=100.0), OdorConcentration(name='Ctrl', concentration_mM=100.0), OdorConcentration(name='TCA', concentration_mM=10.0), OdorConcentration(name='GCA', concentration_mM=10.0), OdorConcentration(name='TDCA', concentration_mM=10.0), OdorConcent

In [4]:
# Step 2: Load or create a sample database
sample_db = SampleDB()

try:
    sample_db.load(db_file_path)
    print("Loaded existing sample database.")
except FileNotFoundError:
    print("Sample database not found. Creating a new one.")
    sample_db.save(db_file_path)  # Create a new CSV file with headers

sample = config.sample
print(sample.id)
existing_sample = sample_db.get_sample(sample.id)



Error loading sample database: 'dof'
Loaded existing sample database.
20220426_RM0008_130hpf_fP1_f3


In [5]:
# Step 3: Check if the sample is already in the database and ask if you want to update it
update=True
if existing_sample:
    print(f"Sample with ID {sample.id} already exists.")
    update = input("Do you want to update the existing sample? (y/n): ").strip().lower() == 'y'
    print(update)
# Ensure initial columns are present
sample_db.add_column('sample_id')
sample_db.add_column('root_path')
sample_db.add_column('config_path')
sample_db.add_column('trials_path')
sample_db.add_column('anatomy_path')
sample_db.add_column('em_path')
sample_db.add_column('update')

# Add the sample data to the database
sample_db.samples[sample.id] = {
    "sample": sample,
    "root_path": root_path,
    "config_path": config_file_path,
    "trials_path": trials_path,
    "anatomy_path": anatomy_path,
    "em_path": em_path,
    "update": update
}


In [6]:
# Step 4: Save the updated database to a file
sample_db.save(db_file_path)
print(f"Sample database saved to {db_file_path}.")

Sample database saved to \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv.


In [7]:
sample_db

SampleDB(sample_ids=['20220426_RM0008_130hpf_fP1_f3'])

In [8]:

# Step 5: Add paths to the configuration and save back to the JSON file if needed
paths = {
    "root_path": root_path,
    "config_path": config_file_path,
    "trials_path": trials_path,
    "anatomy_path": anatomy_path,
    "em_path": em_path
}

updated_config = update_experiment_config(config, paths)
save_experiment_config(updated_config, config_file_path)


In [1]:
# Step 1: Load an experiment configuration from a JSON file

import os
import glob
from scripts.sample_db import SampleDB
from scripts.config_model import Experiment, load_experiment_config, save_experiment_config, update_experiment_config




# Inputs
# Load database csv file
db_file_path = (r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\mo'
                r'ntruth\sample_db.csv')

# root path 
root_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3'

# Loading experiment configuration
config_file_path = glob.glob(os.path.join(root_path, 'config_*.json'))[0]
print(config_file_path)

# Define your directories here
trials_path = os.path.join(root_path, "trials")
anatomy_path = os.path.join(root_path, "anatomy")
raw_trials_path = os.path.join(trials_path, "raw")
em_path = None  # Set to None if not applicable

config = load_experiment_config(config_file_path)
print("Loaded experiment configuration:")
print(config)

# Step 2: Load or create a sample database
sample_db = SampleDB()

try:
    sample_db.load(db_file_path)
    print("Loaded existing sample database.")
except FileNotFoundError:
    print("Sample database not found. Creating a new one.")
    sample_db.save(db_file_path)  # Create a new CSV file with headers

sample = config.sample
existing_sample = sample_db.get_sample(sample.id)

# Step 3: Check if the sample is already in the database and ask if you want to update it
update = True
if existing_sample:
    print(f"Sample with ID {sample.id} already exists.")
    update = input("Do you want to update the existing sample? (y/n): ").strip().lower() == 'y'
    print(update)

# Ensure initial columns are present
sample_db.add_column('sample_id')
sample_db.add_column('root_path')
sample_db.add_column('config_path')
sample_db.add_column('trials_path')
sample_db.add_column('anatomy_path')
sample_db.add_column('em_path')
sample_db.add_column('update')

# Add the sample data to the database
sample_db.samples[sample.id] = {
    "sample": sample,
    "root_path": root_path,
    "config_path": config_file_path,
    "trials_path": trials_path,
    "anatomy_path": anatomy_path,
    "em_path": em_path,
    "update": update
}

# Step 4: Save the updated database to a file
sample_db.save(db_file_path)
print(f"Sample database saved to {db_file_path}.")

# Step 5: Add paths to the configuration and save back to the JSON file if needed
paths = {
    "root_path": root_path,
    "config_path": config_file_path,
    "trials_path": trials_path,
    "anatomy_path": anatomy_path,
    "em_path": em_path
}

updated_config = update_experiment_config(config, paths)
save_experiment_config(updated_config, config_file_path)



\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\config_20220426_RM0008_130hpf_fP1_f3.json
Loaded experiment configuration:
sample=Sample(id='20220426_RM0008_130hpf_fP1_f3', parents_id='P1', genotype='aTubulin:GCamp6s x GAD1b:DsRed', phenotype='positive expression, nacre', dof='21.04.2022-09:30', hpf=130, body_length_mm=4) params_odor=ParamsOdor(odor_list=['Ala', 'Ser', 'Ctrl', 'TCA', 'GCA', 'TDCA', 'Cad', 'SA'], odor_sequence=['Ala', 'Ser', 'Ctrl', 'TCA', 'GCA', 'TDCA', 'Cad', 'SA', 'Cad', 'TDCA', 'GCA', 'TCA', 'Ser', 'Ctrl', 'Ala', 'SA', 'Ctrl', 'Cad', 'Ala', 'Ser', 'TDCA', 'GCA', 'TCA', 'SA'], odor_concentration_uM=[OdorConcentration(name='Ala', concentration_mM=100.0), OdorConcentration(name='Ser', concentration_mM=100.0), OdorConcentration(name='Ctrl', concentration_mM=100.0), OdorConcentration(name='TCA', concentration_mM=10.0), OdorConcentration(name='GCA', concentration_mM=10.0), OdorConcentration(name='TDCA', concentration_mM=10.0), OdorConcent